### LLM CHAIN

In [2]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


# Load API key
load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.2)


prompt = PromptTemplate.from_template("Write a short sentence about {topic}.")

chain = LLMChain(llm=llm, prompt=prompt)

result = chain.invoke({"topic": "langchain"})
print(result["text"])


C:\Users\admin\AppData\Local\Temp\ipykernel_31400\2703165350.py:17: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


Langchain is a framework for developing applications powered by language models.


### RETRIEVAL QA CHAIN

In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.schema import Document

# Documents
docs = [
    Document(page_content="LangChain is a library for building LLM applications."),
    Document(page_content="Gemini is Google’s latest large language model.")
]

#Embedding
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

# Vector store
vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever()

# RAG chain
retriever_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=False
)

result = retriever_chain.invoke({"query": "What is LangChain?"})
print(result["result"])


LangChain is a library for building LLM applications.


### RETRIEVAL QA WITH SOURCES CHAIN

In [9]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.schema import Document

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

docs = [
    Document(page_content="LangChain supports RAG pipelines.",metadata={"source": "doc1"}),
    Document(page_content="Agents help connect LLMs to tools.",metadata={"source": "doc2"})
]

db = FAISS.from_documents(docs, embeddings)
retriever = db.as_retriever()

qa = RetrievalQAWithSourcesChain.from_llm(
    llm=llm,
    retriever=retriever
)

result = qa.invoke({"question": "What does LangChain support?"})

print("ANSWER:", result["answer"])
print("SOURCES:", result["sources"])


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 Resource exhausted. Please try again later. Please refer to https://cloud.google.com/vertex-ai/generative-ai/docs/error-code-429 for more details..


ANSWER: LangChain supports RAG pipelines.

SOURCES: doc1


### SEQUENTIAL CHAIN

In [16]:
import os
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

# 1. Setup (assuming GOOGLE_API_KEY is set in environment)
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.2)
parser = StrOutputParser()

# --- STEP 1: Translator Chain ---
# Input: original_text
# Output: translated_text
translate_prompt = PromptTemplate.from_template("Translate the following text into Tamil: \n\n{original_text}")
translator_chain = translate_prompt | llm | parser

# --- STEP 2: Summarizer Chain ---
# Input: translated_text (which comes from the translator_chain output)
# Output: final_summary
summarize_prompt = PromptTemplate.from_template("You are an expert Tamil summarizer. Summarize the following Tamil text in one sentence: \n\n{translated_text}")
summarizer_chain = summarize_prompt | llm | parser

# --- Sequential Chain with LCEL ---
# The output of 'translator_chain' automatically becomes the input for 'summarizer_chain'
# Note the use of the '|' operator.

 # Define the initial input for the translator chain
 # Pass the output of the translator_chain to the next prompt's input variable 'translated_text'
sequential_chain = {"original_text": lambda x: x['text'] } | translator_chain | {  "translated_text": lambda x: x} | summarizer_chain
#sequential_chain =  translator_chain | summarizer_chain

# --- Invoke the Chain ---
input_text = "I love LangChain and Gemini because they allow me to build incredible applications very quickly."

response = sequential_chain.invoke({"text": input_text})

print(f"Original Text: {input_text}")

print(f"Final Summarized Response (in Tamil): {response}")

Original Text: I love LangChain and Gemini because they allow me to build incredible applications very quickly.
Final Summarized Response (in Tamil): லாங் செயின் மற்றும் ஜெமினி என்னை நம்பமுடியாத பயன்பாடுகளை மிக விரைவாக உருவாக்க அனுமதிப்பதால் எனக்கு மிகவும் பிடிக்கும்.


In [6]:
import os
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

# --- 1. Setup ---
# Ensure your GOOGLE_API_KEY is set in your environment
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.3)
parser = StrOutputParser()

# --- 2. Define the First Chain: Idea Generator ---
# This chain takes 'topic' as input and outputs a product 'idea'.
idea_prompt = PromptTemplate.from_template(
    "Generate a unique and creative product idea based on the following topic: {topic}"
)
idea_chain = idea_prompt | llm | parser

# --- 3. Define the Second Chain: Slogan Creator ---
# This chain takes the 'idea' (the output of the first chain) as input and outputs a 'slogan'.
slogan_prompt = PromptTemplate.from_template(
    "Write a short, catchy, and inspiring slogan for this product: {idea}"
)
slogan_chain = slogan_prompt | llm | parser

# --- 4. Combine Chains Sequentially using LCEL ---
# The output of 'idea_chain' is automatically passed as the input to 'slogan_chain'.
# The initial input key is 'topic'.
# The final output is the slogan from the last step.
sequential_chain = idea_chain | slogan_chain

# --- 5. Invoke the Chain ---
input_topic = "eco-friendly water bottles"
print(f"Input Topic: {input_topic}\n")

# Run the entire pipeline with a single invocation
response = sequential_chain.invoke({"topic": input_topic})

print("--- Sequential Chain Result ---")
print(f"Final Slogan: {response}")

Input Topic: eco-friendly water bottles

--- Sequential Chain Result ---
Final Slogan: Here are a few slogan options for the "Seedling Sip," playing with different angles:

**Short & Catchy:**

*   **Seedling Sip: Drink, Plant, Grow.** (Simple, action-oriented)
*   **Sip Sustainably. Grow Beautifully.** (Highlights both aspects)
*   **The Bottle That Blooms.** (Memorable and poetic)
*   **Seedling Sip: Water Today, Flowers Tomorrow.** (Focuses on the future)
*   **Plant the Change. One Sip at a Time.** (Emphasizes impact)

**Inspiring & Benefit-Oriented:**

*   **Seedling Sip: Quench Your Thirst, Grow the Future.** (Connects personal action to global impact)
*   **Give Back to the Earth, One Sip at a Time.** (Focuses on environmental responsibility)
*   **Seedling Sip: Where Hydration Meets Regeneration.** (Highlights the unique combination)
*   **Drink Well. Plant Hope. Seedling Sip.** (Evokes positive emotions)
*   **The Cycle of Life, in Every Sip.** (More philosophical and thought-

### Map-Reduce Summarization Chain

In [18]:
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 2. Long text
text = """
LangChain is a powerful open-source framework that helps developers 
build applications using large language models. It supports memory, 
agents, retrieval-augmented generation, document processing, and more. 
The framework is widely used for chatbots, automation, RAG systems, 
workflow orchestration, and AI-driven enterprise applications.
"""

# 3. Convert into Documents
docs = [Document(page_content=text)]

# 4. Split into chunks (Map phase)
splitter = RecursiveCharacterTextSplitter(chunk_size=150, chunk_overlap=20)
split_docs = splitter.split_documents(docs)

print("\nSplitter",splitter)
print("\nSplit documet",split_docs)

# 5. Map-Reduce chain
chain = load_summarize_chain(
    llm,
    chain_type="map_reduce" # chain_type="stuff"
)

# 6. Run summarizer
summary = chain.invoke({"input_documents": split_docs})

print(" \nFINAL SUMMARY:\n")
print(summary["output_text"])



Splitter <langchain_text_splitters.character.RecursiveCharacterTextSplitter object at 0x000001BDE0165480>

Split documet [Document(metadata={}, page_content='LangChain is a powerful open-source framework that helps developers \nbuild applications using large language models. It supports memory,'), Document(metadata={}, page_content='agents, retrieval-augmented generation, document processing, and more. \nThe framework is widely used for chatbots, automation, RAG systems,'), Document(metadata={}, page_content='workflow orchestration, and AI-driven enterprise applications.')]
 
FINAL SUMMARY:

LangChain is an open-source framework that helps developers build LLM-powered applications like chatbots using tools for memory, automation, document processing, and RAG. It also supports AI-driven enterprise applications and workflow orchestration.


### STUFF SUMMARIZATION

In [17]:
# LCEL style

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema.runnable import RunnableMap, RunnableLambda
from langchain.schema import Document

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

# Combine text
def combine_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

# LCEL pipeline
stuff_chain = (
    RunnableMap({"text": combine_docs})| (lambda x: f"Summarize this:\n\n{x['text']}")| llm
)

docs = [Document(page_content="LangChain simplifies LLM workflow engineering...")]

summary = stuff_chain.invoke(docs)

print(summary.content)


LangChain streamlines the process of building applications powered by large language models (LLMs) by providing tools and abstractions that make it easier to manage and connect different components like data sources, models, and prompts.
